# tokenizer的学习，代码涉及huggingface-tokenizers库和tansformers中的tokenizer

## 一.基础
1. 简介：tokenizer是分词器，在我们要把句子输入到模型时，模型只能接收数字的输入（比如input_ids一些数字序列）才能计算，所以tokenizer就是把原始一句人类语言变成模型能够接收的输入，而广义上的tokenizer是把句子分成一个token的序列，这也是整个过程中最重要的部分，其实也没错，毕竟把句子分成token后，然后可以通过预先的vocab.txt的token2id把token映射成数字
2. tokenizers这个库是使用rust实现，速度很快;使用tokenizers这个库里面的Tokenizer类的可以使用ailgn-track，对齐追踪，找到每个处理后的token在原始文本中的位置，
3. tokenizers这个库里面的主要代码是 site-packages/tokenizers/tokenizers.cpython-310-x86_64-linux-gnu.so这个文件实现的所有细节，其他只有pyi文件说明
4. 如果在huggingface模型中用到了tokenizers这个库，那相对应的tokenizer就是fast-tokenizer（使用rust加速，可以追踪token位置），所以如果一个载入的模型使用的不是这个Tokenizer类，那它就可能不能追踪token位置


## 二.创建一个Tokenizer类

![图片](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter6/tokenization_pipeline.svg)

### 1. normalization 文本规范化：把不规范的文本变成规范的文本
> * 如果遇到的是html，最好在html前处理成句子形式，记录下offset映射，然后再进行normalization
> * Normalizer是基类，主要用到normalize_str方法规范化文本，还有normalize追踪（不常用）
> * 规范化的使用：组合多个处理
> * unicode编码：用NFKD
>> Unicode Normalization： https://xobo.org/unicode-normalization-nfd-nfc-nfkd-nfkc/
>> 
>> unicode中对于同一文本的不同表示会有不同的unicode，所以会对相同意义的不同形式的unicode进行等价处理，比如‘字的Unicode值\u2F4’ 和 ‘方字Unicode值是\u65B9 ’；等价分为：标准等价和兼容等价
>> 
>> 标准等价：保持视觉上和功能上等价。例如 '≠' (\u2260) 标准等价于 '=' 和 '̸'(\u003d\u0338)的组合。区别在于一个是一个字符，一个是两个字符。
>> 
>> 兼容等价：兼容等价比标准等价范围更广，也更关注纯文字的等价，并把一些语义上的不同形式归结在一起。如果字符是标准等价，那么他们一定是兼容等价，反之就不一定了。例如 '㊋１ａ' (\u328b\uff11\uff41) 带圆圈的火，全角的数字和字母兼容等价与 '火1a' (\u706b\u0031\u0061)。
>> 
>> 四种组合，此基础之后Unicode又定义了两种形式，完全合成，完全分解。由此组合出四种形式: NFC、NFD、NFKC、NFKD。
>> 
>> C 代表 Composition 就是组合的意思
>> 
>> D 代表 Decomposition 就是分解的意思。
>> 
>> K 代表 Compatibility 就是兼容等价模式，如果不带K就是标准等价。
>>
>> 分解和组合主要用于是字母符号。汉字我没找到可以分解和组合的例子。汉字的兼容等价场景就很多，除了我遇到的部首⽅，还有 ㊥ 与 中、 ㍿ 与 株式会社、全角与半角。。。


In [1]:
from tokenizers.normalizers import BertNormalizer,Lowercase,NFC,NFD,NFKC,NFKD,Nmt,Normalizer,Precompiled,Replace,Sequence,Strip,StripAccents

In [2]:
s = '  /t \n HDdfé㊋1234二五七二\n'
print(NFC().normalize_str(s))
print(NFD().normalize_str(s))
print(NFKC().normalize_str(s))
print(NFKD().normalize_str(s))
print(Lowercase().normalize_str(s))  # 小写
print(Strip().normalize_str(s))  # strip

print(StripAccents().normalize_str(s)) 
print(StripAccents().normalize_str(NFKD().normalize_str(s)))  # 删除 unicode 中的所有重音符号（与 NFD 一起使用以保持一致性）

print(Replace("f", "k").normalize_str(s)) # 规则替代
from tokenizers import Regex
print(Replace(Regex("\d+"), '[NUM]').normalize_str(s)) # 规则可以用regex，要替代的只能是字符串

print(BertNormalizer(clean_text=True,handle_chinese_chars=True,strip_accents=True,lowercase=True).normalize_str(s))
# bert处理，clean_text：移除 control characters字符并且用经典字符替代各种空格(比如\n),处理中文，在各个字中间添加空格，去除重音，

print(Sequence([NFD(), StripAccents()]).normalize_str(s)) # 组合多个处理

  /t 
 HDdfé㊋1234二五七二

  /t 
 HDdfé㊋1234二五七二

  /t 
 HDdfé火1234二五七二

  /t 
 HDdfé火1234二五七二

  /t 
 hddfé㊋1234二五七二

/t 
 HDdfé㊋1234二五七二
  /t 
 HDdfé㊋1234二五七二

  /t 
 HDdfe火1234二五七二

  /t 
 HDdké㊋1234二五七二

  /t 
 HDdfé㊋[NUM]二五七二

  /t   hddfe㊋1234 二  五  七  二  
  /t 
 HDdfe㊋1234二五七二



### 2. pre-tokenization 预标记化：把一整句话先按简单规则切分成token序列，保证模型输入token的最大元素存在
> * 预标记化是将文本拆分为更小的对象的行为，这些对象给出了训练结束时标记的上限。一般实体识别需要这个，
> * 和model-tokenization不同的是，pre-tokenization就是用简单的规则对文本进行拆分达到每个token都有一个完整label的状态（实体识别）来确保模型不会有多个token组合再一起的token，比如你不想token中包含一个空格，这个时候就可以用pretokenizer，而model-tokenization需要将每个token拆分达到能够得到id的状态
> * 使用pre_tokenize_str方法,它能够得到拆分后每个token的位置offset
> * 使用预分词器把单词拆分，来确保最终模型得到的token不能大于预分词器的token。比如’it is‘ 经常出现在一起，我们最后得到的结果不需要这样的，所以可以用空格分词预先拆分这个，如果没有将我们的输入拆分为单词的预标记器，我们可能会得到与多个单词重叠的标记，比如‘it is’

In [3]:
from tokenizers.pre_tokenizers import Digits,ByteLevel,Whitespace,WhitespaceSplit,Punctuation,Metaspace,CharDelimiterSplit,Split,Sequence

In [4]:
s = 'Hello my friend1, how are you23?'
print(ByteLevel().pre_tokenize_str(s))  # 以空格拆分，并把所有bytes映射到可见字符上，对于非ascii字符会不可读但是有效，
print(Whitespace().pre_tokenize_str(s)) # 使用\w+|[^\w\s]+ 进行拆分
print(WhitespaceSplit().pre_tokenize_str(s)) # 使用空格拆分

print(Punctuation().pre_tokenize_str(s)) # 使用标点拆分
print(Metaspace().pre_tokenize_str(s)) # 拆分空格并用特殊字符“__”替换它们 (U+2581)
print(CharDelimiterSplit('r').pre_tokenize_str(s)) # 以给定字符进行拆分
print(Digits().pre_tokenize_str(s)) # 从任何其他字符中拆分出数字。

# 以规则进行拆分，pattern是字符串或规则，是指拆分出来的token，这些token可以有各种行为
# behaviror必须removed，isolated，merged_with_previous，merged_with_next。contiguous中的一个，
# invert should be a boolean flag.
print(Split(pattern = 'are', behavior = "isolated", invert = False).pre_tokenize_str(s)) # token独立
print(Split(pattern = 'are', behavior = "merged_with_previous", invert = False).pre_tokenize_str(s)) # token和前一个组合 
print(Split(pattern = 'are', behavior = "merged_with_next", invert = False).pre_tokenize_str(s)) # token和后一个组合
print(Split(pattern = 'are', behavior = "removed", invert = False).pre_tokenize_str(s)) # token移除
print(Split(pattern = 'are', behavior = "contiguous", invert = False).pre_tokenize_str(s)) #token连续
from tokenizers import Regex
print(Split(pattern = Regex("\d+"), behavior = "contiguous", invert = False).pre_tokenize_str(s)) #token连续


print(Sequence([Punctuation(), WhitespaceSplit()]).pre_tokenize_str(s)) # 组合pre-tokenizer



[('ĠHello', (0, 5)), ('Ġmy', (5, 8)), ('Ġfriend', (8, 15)), ('1', (15, 16)), (',', (16, 17)), ('Ġhow', (17, 21)), ('Ġare', (21, 25)), ('Ġyou', (25, 29)), ('23', (29, 31)), ('?', (31, 32))]
[('Hello', (0, 5)), ('my', (6, 8)), ('friend1', (9, 16)), (',', (16, 17)), ('how', (18, 21)), ('are', (22, 25)), ('you23', (26, 31)), ('?', (31, 32))]
[('Hello', (0, 5)), ('my', (6, 8)), ('friend1,', (9, 17)), ('how', (18, 21)), ('are', (22, 25)), ('you23?', (26, 32))]
[('Hello my friend1', (0, 16)), (',', (16, 17)), (' how are you23', (17, 31)), ('?', (31, 32))]
[('▁Hello', (0, 5)), ('▁my', (5, 8)), ('▁friend1,', (8, 17)), ('▁how', (17, 21)), ('▁are', (21, 25)), ('▁you23?', (25, 32))]
[('Hello my f', (0, 10)), ('iend1, how a', (11, 23)), ('e you23?', (24, 32))]
[('Hello my friend', (0, 15)), ('1', (15, 16)), (', how are you', (16, 29)), ('23', (29, 31)), ('?', (31, 32))]
[('Hello my friend1, how ', (0, 22)), ('are', (22, 25)), (' you23?', (25, 32))]
[('Hello my friend1, how are', (0, 25)), (' you23?

### 3. model 模型分词：对之前分好的每个token序列再就行单个token的切分，一般是subword用到
> * 它是实际分词的核心算法，是针对模型的，所以如果使用Tokenizer，一定要传入这个参数
> * 模型的作用是使用它学到的规则将你的“单词”拆分成标记。它还负责将这些标记映射到模型词汇表中相应的 ID。
> * 它可以通过语料库进行训练，不同方法需要的参数不一样
> * 基本方法：get_trainer，id_to_token，save，token_to_id，tokenize(输入token的字符串，输出A List of Token)，from_file,read_file
> * 它的tokenize方法输入的是一个token，而不是一句话，把token进行切分和id映射操作
> * unk_token需要被vocab包含
> * 如果要使用，用BPE（频率）或WordPiece（语言概率），所以如果有训练集，用wordpiece感觉可能更好，但是这个可能更依赖数据集，数据集改变，效果迅速拉跨？而BPE可能更通用一些，但在特殊数据集上的效果可能不如wordpiece
>> * https://zhuanlan.zhihu.com/p/191648421
>> * https://huggingface.co/docs/transformers/tokenizer_summary#bytepair-encoding-bpe
>> * WordLevel: 词基本的方法，就是把每个token映射到一个id
>> * BPE : subword-tokenization 算法：从字符开始，同时将最常见的字符组合在一起，从而创建新的token，然后迭代创建token，BPE 能够通过使用多个子词标记来构建它从未见过的词，因此需要更小的词汇表，具有“unk”（未知）标记的机会更少。,merges (List[Tuple[str, str]], optional) — A list of pairs of tokens,首先将词分成单个字符，然后依次用另一个字符替换频率最高的一对字符 ，直到循环次数结束,这个要保证merge里面的每个token都在vocab中且合并之后的token也要在vocab中
>> * Google的Bert模型在分词的时候使用的是WordPiece算法。与BPE算法类似，WordPiece算法也是每次从词表中选出两个子词合并成新的子词。与BPE的最大区别在于，如何选择两个子词进行合并：BPE选择频数最高的相邻子词合并，而WordPiece选择能够提升语言模型概率最大的相邻子词加入词表。使用著名的## 前缀来识别作为单词一部分的标记（即不是单词的开头）

In [5]:
from tokenizers.models import BPE,WordLevel,WordPiece,Unigram
from tokenizers import Tokenizer
import json
token_list = 'hello my friend'.split()
vocab  = ['he','llo','l','ll','o','hello','my','friend','[UNK]']
vocab = {j:i for i,j in enumerate(vocab)}
with open('./test_vocab.json', 'w', encoding='utf-8') as file:
    json.dump(vocab, file, ensure_ascii=False, indent=4)


In [6]:
print([(i.value,i.id,i.offsets)for i in WordLevel(vocab=vocab,unk_token="[UNK]").tokenize('hello')])
print([(i.value,i.id,i.offsets)for i in BPE(vocab=vocab,merges=[('l','l'),('ll','o')],unk_token="[UNK]").tokenize('hello')])

print([(i.value,i.id,i.offsets)for i in WordPiece(vocab=vocab,unk_token="[UNK]").tokenize('hello')])

# uni_vocab = [("hello", 0.2442),("my", -0.2342),('[UNK]',-3)]
# print([(i.value,i.id,i.offsets)for i in Unigram(vocab=uni_vocab).tokenize('hello')])


[('hello', 5, (0, 5))]
[('[UNK]', 8, (0, 1)), ('[UNK]', 8, (1, 2)), ('llo', 1, (2, 5))]
[('hello', 5, (0, 5))]


### 4. postprocessor 后处理：把模型分词好的token序列再做一些处理，比如加上一些特殊的token，cls之类的
> * 请注意，与预分词器或规范器相反，您无需在更改后处理器后重新训练分词器。
> * 模型输入模板化,使用特殊字符来处理句子，一般使用句子模板,$A表示第一个句子,$B第二个句子。,:1 想要输入的每个部分，句子的索引：默认为 0 表示所有内容（这就是为什么我们没有 $A:0），



In [7]:
from tokenizers.processors import TemplateProcessing,BertProcessing,RobertaProcessing,ByteLevel
post_processs = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[("[CLS]", 1), ("[SEP]", 2)],
)

### 5. tokenizer：根据上面的组件组成一个Tokenizer
> * Tokenizer必须含有model-mokenizer，Tokenizer可以随时变更normalization，pre-tokenizer,post-processor

In [8]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
bert_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))

from tokenizers import normalizers
from tokenizers.normalizers import NFD, Lowercase, StripAccents
bert_tokenizer.normalizer = normalizers.Sequence([NFD(), Lowercase(), StripAccents()])

from tokenizers.pre_tokenizers import Whitespace
bert_tokenizer.pre_tokenizer = Whitespace()

from tokenizers.processors import TemplateProcessing
bert_tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", 1),
        ("[SEP]", 2),
    ],
)


## 三.使用构建好的Tokenizer

In [9]:
s = "Hello, ! How are you 😁 ?"
# 载入模型的tokenizer
from transformers import AutoTokenizer
checkpoint = "/large_files/5T/huggingface_cache/pretrained_model/microsoft--BiomedNLP-PubMedBERT-base-uncased-abstract/"
model_cache_dir = "/large_files/5T/huggingface_cache/model"
bert_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=checkpoint,
                                               cache_dir=model_cache_dir)._tokenizer
# 自定义的tokenizer


/home/zyl/disk/poetry_env/algorithms-ai-IgVB5U1f-py3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. encode 编码：把原始文本映射成模型可以接收的输入，主要是ids

In [10]:
from tokenizers import Encoding
n_s = bert_tokenizer.normalizer.normalize_str(s)
pt_s = bert_tokenizer.pre_tokenizer.pre_tokenize_str(n_s)
m_s = [bert_tokenizer.model.tokenize(i[0]) for i in pt_s]
print(n_s)
print(pt_s)
print(m_s)
a = Encoding()
all_m_s = [j for i in m_s for j in i]
print(all_m_s)
# a.tokens = [i for i in all_m_s]
# a.values = [i.value for i in all_m_s]
print(dir(a))
bert_tokenizer.post_processor.process(a)  # 未知代码,而且其中还涉及到pad，truncation等操作

hello, ! how are you 😁 ?
[('hello', (0, 5)), (',', (5, 6)), ('!', (7, 8)), ('how', (9, 12)), ('are', (13, 16)), ('you', (17, 20)), ('😁', (21, 22)), ('?', (23, 24))]
[[<tokenizers.Token object at 0x7fc2a03a9700>, <tokenizers.Token object at 0x7fc2a03a88f0>], [<tokenizers.Token object at 0x7fc2a03a8670>], [<tokenizers.Token object at 0x7fc2a03a8da0>], [<tokenizers.Token object at 0x7fc2a03a8a80>], [<tokenizers.Token object at 0x7fc2a03a8850>], [<tokenizers.Token object at 0x7fc2a03a8e90>], [<tokenizers.Token object at 0x7fc2109ce3d0>], [<tokenizers.Token object at 0x7fc2109ce420>]]
[<tokenizers.Token object at 0x7fc2a03a9700>, <tokenizers.Token object at 0x7fc2a03a88f0>, <tokenizers.Token object at 0x7fc2a03a8670>, <tokenizers.Token object at 0x7fc2a03a8da0>, <tokenizers.Token object at 0x7fc2a03a8a80>, <tokenizers.Token object at 0x7fc2a03a8850>, <tokenizers.Token object at 0x7fc2a03a8e90>, <tokenizers.Token object at 0x7fc2109ce3d0>, <tokenizers.Token object at 0x7fc2109ce420>]
['__cla

Encoding(num_tokens=2, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [11]:
print(bert_tokenizer.encode(s))
print(bert_tokenizer.encode_batch([s]*2))

Encoding(num_tokens=11, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
[Encoding(num_tokens=11, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=11, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]


### 2. decode 编码：把模型的输入ids解码成原始文本
> * 首先将 ID 转换回标记（使用标记器的词汇表）并删除所有特殊标记，然后用空格连接这些标记
> * 解码器和编码器要对应,虽然它也可以随便换，如果您使用的模型添加了特殊字符来表示给定“单词”的子标记（如 WordPiece 中的“##”），您将需要自定义解码器以正确处理它们

In [12]:
from tokenizers.decoders import BPEDecoder,ByteLevel,Metaspace,WordPiece,CTC
print(bert_tokenizer.decoder)
print(bert_tokenizer.decode(bert_tokenizer.encode(s).ids))
print(bert_tokenizer.decode_batch([bert_tokenizer.encode(s).ids]*2))

bert_tokenizer.decoder = BPEDecoder()
print(bert_tokenizer.decode(bert_tokenizer.encode(s).ids))

hello,! how are you?
['hello,! how are you?', 'hello,! how are you?']
hel##lo,!howareyou?


### 3. 其他

> * add_special_tokens:如果这些标记已经是词汇表的一部分，它只会让 Tokenizer 知道它们。如果它们不存在，则 Tokenizer 会创建它们，并给它们一个新的 id。这些特殊标记永远不会被模型处理（即不会被拆分成多个标记），并且可以在解码时将它们从输出中移除。
> * add_tokens仅当词汇表中不存在给定标记时才添加它们。然后每个令牌都会获得一个新的属性 ID。

In [13]:
bert_tokenizer.enable_padding(direction = 'right',pad_id = 0,pad_type_id = 0,pad_token = '[PAD]',length = None,pad_to_multiple_of = None)
bert_tokenizer.enable_truncation(max_length=512,stride = 0,strategy = 'longest_first',direction = 'right' )
bert_tokenizer.add_special_tokens(["<e1>", "</e1>", "<e2>", "</e2>"])
bert_tokenizer.add_tokens(["<e2>", "</e4>", "<e3>", "</e2>"])

2

## 四.训练定义好的tokenizer
> * 训练tokenizer是一个统计过程，它试图识别给定语料库中最适合选择的子词，用于选择它们的确切规则取决于标记化算法。它是确定性的，这意味着在同一语料库上使用相同的算法进行训练时，总是得到相同的结果。

### 1. 选择一个tokenizer训练器
> * 可以自定义一个tokenizer或者载入一个tokenizer

In [14]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer

In [18]:
from tokenizers import pre_tokenizers,decoders,normalizers
wordpiece_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
wordpiece_tokenizer.normalizer = normalizers.NFKC()
wordpiece_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
wordpiece_tokenizer.decoder = decoders.ByteLevel()

# checkpoint = "/large_files/5T/huggingface_cache/pretrained_model/microsoft--BiomedNLP-PubMedBERT-base-uncased-abstract/"
# model_cache_dir = "/large_files/5T/huggingface_cache/model"
# wordpiece_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=checkpoint,
#                                                cache_dir=model_cache_dir).backend_tokenizer

wordpiece_tokenizer.get_vocab_size()

0

In [ ]:
from tokenizers import Tokenizer
token_list = 'Hello my friend'.split()
vocab  = ['hello','my','friend','[UNK]']
vocab  = {j:i for i,j in enumerate(vocab)}
print(vocab)
WordPiece(vocab=vocab,unk_token="[UNK]").tokenize('hello my')

### 2. 选择tokenizer-model对应的trainer
> * 注意trainer和tokenizer对应。每个trainer可以设置一些特定的参数

In [19]:
wordpiece_trainer =  WordPieceTrainer(vocab_size=90,min_frequency=3,
                     special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

### 3. 配置要使用的预料
> * train_from_iterator需要数据是字符串列表或者迭代的字符串
> * train需要的是一些文件

In [20]:
data = [
    "Beautiful is better than ugly.",
    "Explicit is better than implicit.",
    "Simple is better than complex.",
    "Complex is better than complicated.",
    "Flat is better than nested.",
    "Sparse is better than dense.",
    "Readability counts.",
]

### 4. 训练

In [21]:
# wordpiece_tokenizer.train_from_iterator(data*20, trainer=wordpiece_trainer)
wordpiece_tokenizer.train_from_iterator(iter(data), trainer=wordpiece_trainer)

# wordpiece_tokenizer.train(files, trainer=wordpiece_trainer)

wordpiece_tokenizer.get_vocab_size()

77

### 5. 保存

In [22]:
wordpiece_tokenizer.save("test_bert_vocab.json")

### 6. 载入训练好的tokenizer并且使用
> * save对应from_file
> * from_pretrained使用hub上的tokenizer.json
> * 使用_tokenizer进行替换

In [23]:
s = "Beautiful is better than ugly."
new_token = Tokenizer.from_file("test_bert_vocab.json")

checkpoint = "/large_files/5T/huggingface_cache/pretrained_model/microsoft--BiomedNLP-PubMedBERT-base-uncased-abstract/"
model_cache_dir = "/large_files/5T/huggingface_cache/model"
bt_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=checkpoint,
                                               cache_dir=model_cache_dir)
print(bt_tokenizer.encode(s))
bt_tokenizer._tokenizer = new_token
print(bt_tokenizer.encode(s))

[2, 1765, 19744, 1733, 1703, 1744, 3387, 1981, 16110, 1716, 17, 3]
[32, 56, 38, 47, 49, 45, 43, 57, 49, 37, 68, 71, 69, 32, 49, 50, 37, 51, 5]


## 五.hugging-face中的tokenizer

###  1.训练
> * huggingface有一个可以训练与现有标记器相同特征的新标记器的转换器：AutoTokenizer.train_new_from_iterator（）
> * 使用from_pretrained和save_pretrained

In [25]:
checkpoint = "/large_files/5T/huggingface_cache/pretrained_model/microsoft--BiomedNLP-PubMedBERT-base-uncased-abstract/"
model_cache_dir = "/large_files/5T/huggingface_cache/model"
old_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=checkpoint,
                                               cache_dir=model_cache_dir)
s = "Beautiful is better than ugly."
print(old_tokenizer.tokenize(s))
new_tokenizer = old_tokenizer.train_new_from_iterator(data, 52000)
print(new_tokenizer.tokenize(s))

#新训练的分词器可以保存起来，注意这里用的是AutoTokenizer
new_tokenizer.save_pretrained( "code-search-net-tokenizer" )


['be', '##aut', '##if', '##ul', 'is', 'better', 'than', 'ug', '##ly', '.']



['beautiful', 'is', 'better', 'than', 'ugly', '.']


('code-search-net-tokenizer/tokenizer_config.json',
 'code-search-net-tokenizer/special_tokens_map.json',
 'code-search-net-tokenizer/vocab.txt',
 'code-search-net-tokenizer/added_tokens.json',
 'code-search-net-tokenizer/tokenizer.json')

###  2.使用
> * tokenizer主要根据一组规则将文本拆分为token，然后在句子中添加些特殊token，然后将所有token映射为数字，然后把这些数字转换成张量
> * call的流程：raw_text-（_tokenizer.tokenize方法）->tokens --> special tokens（由prepare_for_model方法添加） -（convert_tokens_to_ids）->input_ids -(prepare_for_model)-->model-(decode)->
> * call的返回,实际是enconding对象，可以返回很多信息，包括return_offsets_mapping：
>> * input_id表示句子中每个tokend的indices映射（convert_tokens_to_ids）
>> * attention_mask是否对应位置的token应该被加入注意力计算（1表示使用注意力机制的时候计算它）,为0的不要计算attention
>> * token_type_ids: 就是当输入是好几句话时表示这个token属于哪句话,主要用于句子对
>> * offset...
>> * encode的时候如果碰到不在vocab中的，就会报错，除非里面有预处理
> * padding:
>> * padding(默认False)：由于各句话总是长度不一致，而模型需要输入的是张量要保证一个batch的长度一样，所以padding使用一个特殊的padding-token（一般0）来补齐短的句子的token，直到和一个batch中最长句子的tokens长度一致
>> * 使用padding时，补齐的token的attention-mask对应的位置为0，
>> * padding默认longest ，可以指定’max_length‘
> * truncation:
>> * （默认False）：句子太长所以截断，使用True表示模型最大可接受的长度
>> *  按模型max_length输入进行截断,默认False，按model_max_length或者输入最大长度或max_length

In [43]:
s = "Beautiful is better than ugly."
print(old_tokenizer(s, return_offsets_mapping=True,padding=True,  max_length=8, truncation=True, return_tensors='pt' ))  # 等于encode_plus,返回的是encodeing对象
print(old_tokenizer.encode_plus(s))
print(dir(old_tokenizer(s)))
print(old_tokenizer(s).word_ids)
print(old_tokenizer.batch_encode_plus([s]*2))

print('#'*30)
print(old_tokenizer.decode([102, 2, 3, 4, 5, 6, 7]))
print(old_tokenizer.convert_ids_to_tokens([102, 2, 3, 4, 5, 6, 7]))
print(old_tokenizer.convert_tokens_to_string(old_tokenizer.convert_ids_to_tokens([102, 2, 3, 4, 5, 6, 7])))

print('#'*30)
print(old_tokenizer.encode(s))
print(old_tokenizer.tokenize(s))
print(old_tokenizer.convert_tokens_to_ids(old_tokenizer.tokenize(s)))

print('#'*30)
print(old_tokenizer._tokenizer.token_to_id("[SEP]"))


print('#'*30)
special_tokens_dict = {'additional_special_tokens': ['[C1]','[C2]','[C3]','[C4]']}
num_added_toks = old_tokenizer.add_special_tokens(special_tokens_dict)
# model.resize_token_embeddings(len(tokenizer))

print('#'*30)
new_tokens = ['token1', 'token2'] 
old_tokenizer.add_tokens(new_tokens)



{'input_ids': tensor([[    2,  1765, 19744,  1733,  1703,  1744,  3387,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]]), 'offset_mapping': tensor([[[ 0,  0],
         [ 0,  2],
         [ 2,  5],
         [ 5,  7],
         [ 7,  9],
         [10, 12],
         [13, 19],
         [ 0,  0]]])}
{'input_ids': [2, 1765, 19744, 1733, 1703, 1744, 3387, 1981, 16110, 1716, 17, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['_MutableMapping__marker', '__abstractmethods__', '__class__', '__class_getitem__', '__contains__', '__copy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__ior__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__or__', '__reduce__'

0

## 其他

In [ ]:

from datasets import load_dataset

wnut = load_dataset("wnut_17")
print(1)
from transformers import AutoTokenizer

# 需要设置is_split_into_words=True将单词标记为子词
# 使用tokenizer的时候如果传入是一个列表，那会把列表中每个元素都看作一个样本，对这个样本进行切分
# 但如果使用is_split_into_words=True，则会把最小单元的列表看作一个样本，而且会对这个样本中的每个token进一步切分
# tokenizer('a')  # 输入一个字符串，则将这个字符串看作一个样本
# {'input_ids': [101, 1037, 102], 'attention_mask': [1, 1, 1]}
# tokenizer('a','b')  # 输入多个字符串，将这些字符串看作一个样本，每个字符串看作一个token
# {'input_ids': [101, 1037, 102, 1038, 102], 'attention_mask': [1, 1, 1, 1, 1]}
# tokenizer(['a'])  # 输入一个字符串的列表，把这个列表看作多个样本，其中每个元素必须是字符串，不能是列表
# {'input_ids': [[101, 1037, 102]], 'attention_mask': [[1, 1, 1]]}
# tokenizer(['a','b']) # 输入一个多个字符串的列表，把这个列表看作多个样本，其中每个元素必须是字符串，不能是列表
# {'input_ids': [[101, 1037, 102], [101, 1038, 102]], 'attention_mask': [[1, 1, 1], [1, 1, 1]]}
# tokenizer([['a'],['b']]) # 输入一个多个列表的列表，报错,除非使用is_split_into_words=True，把里面的每个列表看作一个样本
# tokenizer([['a'],['b']],is_split_into_words=True)
# {'input_ids': [[101, 1037, 102], [101, 1038, 102]], 'attention_mask': [[1, 1, 1], [1, 1, 1]]}
# tokenizer(['a'],['b']) # 输入是多个列表，把每个列表看作一个样本，
# {'input_ids': [[101, 1037, 102, 1038, 102]], 'attention_mask': [[1, 1, 1, 1, 1]]}
# tokenizer(['a','c'],['b','d']) # 输入是多个列表，把这些列表看作多个样本，
# {'input_ids': [[101, 1037, 102, 1038, 102], [101, 1039, 102, 1040, 102]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

# 所以要输入单个样本
# tokenizer('a')
# tokenizer('a','b')

# 如果要输入多个样本，每个元素是一个样本
# tokenizer(['a','b'])
# tokenizer(['a','c'],['b','d'])

# 如果单个样本是list[tokens]
# 如果使用tokenizer(example["tokens"])或者 tokenizer(*[example["tokens"]])，则会把这个列表中每个token看作一个样本，显然不对，
# 如果在外面包裹一个【】，则显然会报错 tokenizer([example["tokens"]])
# 所以需要使用is_split_into_words=True ，这个参数会把最小的列表元素看作是一个样本
# tokenizer(example["tokens"],is_split_into_words=True) # 传入一个样本，tokenizer把这个token列表样本看作一个样本，而且会对里面每个token进行切分
# {'input_ids': [101, 1030, 2703, 17122, 2009, 1005, 1055, 1996, 3193, 2013, 2073, 1045, 1005, 1049, 2542, 2005, 2048, 3134, 1012, 3400, 2110, 2311, 1027, 9686, 2497, 1012, 3492, 2919, 4040, 2182, 2197, 3944, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
# tokenizer([example["tokens"],example["tokens"]],is_split_into_words=True) # 传入两个样本，tok
# {'input_ids': [[101, 1030, 2703, 17122, 2009, 1005, 1055, 1996, 3193, 2013, 2073, 1045, 1005, 1049, 2542, 2005, 2048, 3134, 1012, 3400, 2110, 2311, 1027, 9686, 2497, 1012, 3492, 2919, 4040, 2182, 2197, 3944, 1012, 102], [101, 1030, 2703, 17122, 2009, 1005, 1055, 1996, 3193, 2013, 2073, 1045, 1005, 1049, 2542, 2005, 2048, 3134, 1012, 3400, 2110, 2311, 1027, 9686, 2497, 1012, 3492, 2919, 4040, 2182, 2197, 3944, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

# tokenizer([example["tokens"],example["tokens"]], is_split_into_words=True)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

example = wnut["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])  # 把id转为token，一般在是decode里面的步骤
print(tokens)
print(1)


# 创建一个函数来重新对齐标记和标签，并将序列截断为不超过 DistilBERT 的最大输入长度
def tokenize_and_align_labels(examples):
    """
    输入
    tokens ： ['@paulwalk', 'It', "'s", 'the', 'view', 'from', 'where', 'I', "'m", 'living', 'for', 'two', 'weeks', '.', 'Empire', 'State', 'Building', '=', 'ESB', '.', 'Pretty', 'bad', 'storm', 'here', 'last', 'evening', '.']
    ner_tags： [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0]

    输出
    input_tokens: ['[CLS]', '@', 'paul', '##walk', 'it', "'", 's', 'the', 'view', 'from', 'where', 'i', "'", 'm', 'living', 'for', 'two', 'weeks', '.', 'empire', 'state', 'building', '=', 'es', '##b', '.', 'pretty', 'bad', 'storm', 'here', 'last', 'evening', '.', '[SEP]']
    input_ids: [101, 1030, 2703, 17122, 2009, 1005, 1055, 1996, 3193, 2013, 2073, 1045, 1005, 1049, 2542, 2005, 2048, 3134, 1012, 3400, 2110, 2311, 1027, 9686, 2497, 1012, 3492, 2919, 4040, 2182, 2197, 3944, 1012, 102]
    labels: [-100, 0, -100, -100, 0, 0, -100, 0, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, -100, 0, 0, 0, 0, 0, 0, 0, 0, -100]
    attention:[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

    """
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        # .word_ids(batch_index=i)直接把input_ids中第batch_index的得到它们的字序号id，比如[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, None]
        previous_word_idx = None
        label_ids = []

        # 把后缀词，比如'##s‘都变成-100，然后保留第一个token的label,特殊字符也是-100 , 这些词在token-cls时候label都为-100
        # 其他如果有标签，则保留标签，所以最后的标签中除了原有bio映射的id，还有特殊字符和后缀映射的-100
        # label:[-100, 0, -100, -100, 0, 0, -100, 0, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, -100, 0, 0, 0, 0, 0, 0, 0, 0, -100]
        # 标签中可以有负数，而如果是负数的话，在计算交叉熵损失时，这个负数并不会映射到某个one-hot向量，从而使得对于label是负数的这个token，它的损失是0
        # 交叉熵损失计算每个token损失后要除以有损失的样本数，如果一个token的label是负数，那么它不会参与计算，也就是说它不算样本

        # 其次把所有 ## 前缀的token变为-100的标签，为什么？
        # 可以降低模型训练难度，如果不是-100，则要加入损失计算中，降低模型学习速度,增加要学习的特征个数，模型复杂度增加，
        # 而且理论上，带##的token应该和前面的那个token看成一个整体，所以计算它们两个只要有一个特征输出就行了，
        # 一般使用首个token作为这一些词的特征，所以通过encoder后，这个首个token的特征应该主要包括这个词的特征以及后面前缀token的特征，
        # 所以尽量把不起作用的token变为无意义标签（特殊字符），能主观合并的token只计算一个特征（前缀词）
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels

    return tokenized_inputs

tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)
print(2)
